In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress warnings and info logs

# Disable oneDNN optimizations
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import logging
import tensorflow as tf
from tensorflow.keras.models import load_model

# Set the environment variable to suppress TensorFlow logs


# Suppress TensorFlow logs at runtime
tf.get_logger().setLevel('ERROR')  # Only show errors

# Optional: Set the logging level for CPU-related logs if needed
logging.getLogger('tensorflow_cpu').setLevel(logging.ERROR)

# Load the model from the saved directory
model = load_model('model.h5', compile=False)

# Use the model for predictions or other tasks
# Example: model.predict(input_data)


In [2]:
import pandas as pd

In [3]:
from keras import Sequential
from keras.callbacks import EarlyStopping

from keras.optimizers import Adam, SGD
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.layers import Lambda, Input, GlobalAveragePooling2D,BatchNormalization
from keras.utils import to_categorical
# from keras import regularizers
from tensorflow.keras.models import Model


from keras.preprocessing.image import load_img

In [4]:
import numpy as np

In [5]:
# from keras.preprocessing.image import load_img


In [6]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input

In [7]:
inception_preprocessor

<function keras.src.applications.inception_v3.preprocess_input(x, data_format=None)>

In [8]:
from keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input

In [9]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input

In [10]:
from keras.applications.nasnet import NASNetLarge, preprocess_input
nasnet_preprocessor = preprocess_input

In [11]:
#function to extract features from the dataset by a given pretrained model
img_size = (331,331,3)

def get_features(model_name, model_preprocessor, input_size, data):

    input_layer = Input(input_size)
    preprocessor = Lambda(model_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    
    #Extract feature.
    feature_maps = feature_extractor.predict(data, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [12]:
def extact_features(data):
    inception_features = get_features(InceptionV3, inception_preprocessor, img_size, data)
    xception_features = get_features(Xception, xception_preprocessor, img_size, data)
    nasnet_features = get_features(NASNetLarge, nasnet_preprocessor, img_size, data)
    inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, img_size, data)

    final_features = np.concatenate([inception_features,
                                     xception_features,
                                     nasnet_features,
                                     inc_resnet_features],axis=-1)
    
    print('Final feature maps shape', final_features.shape)
    
    #deleting to free up ram memory
    
    
    
    return final_features

In [13]:
labels = pd.read_csv('data/labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [14]:
classes = sorted(list(set(labels['breed'])))

In [15]:
img_g = load_img('data/test/ffd87c3e44faa0e3ee5fbbdc4c63b59b.jpg',target_size = img_size)
img_g = np.expand_dims(img_g, axis=0)

In [16]:
test_features = extact_features(img_g)



predg = model.predict(test_features)
print(f"Predicted label: {classes[np.argmax(predg[0])]}")
print(f"Probability of prediction): {round(np.max(predg[0])) * 100} %")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Feature maps shape:  (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
Feature maps shape:  (1, 2048)
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Feature maps shape:  (1, 4032)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Feature maps shape:  (1, 1536)
Final feature maps shape (1, 9664)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted label: dhole
Probability of prediction): 100 %
